## The objective of this notebook is to determine what fields make a better job at uniquely identifying the rows or specimens in biocollections

In [39]:
import pandas as pd
from pyspark.sql.functions import isnull, when, count

In [40]:
dataset = spark.read.parquet("./preston-amazon/data-processed/core.parquet")
# dataset = dataset.fillna('')
N = dataset.count()
print("Number of rows: " + str(N))

Number of rows: 3858


In [41]:
df = pd.DataFrame( columns = ['term', 'n_val', 'rate_val', 'n_uniq', 'rate_uniq'] )


In [42]:
i = 0
for c in dataset.columns:
    column_name = "`" + c + "`"
    n_v = dataset.select( count(when(isnull(column_name), column_name)).alias(column_name) ).collect()[0][column_name]
    n_v = N - n_v
    n_c = dataset.select( column_name ).distinct().count()
    df.loc[ i ] = [column_name, n_v, n_v/N, n_c, n_c/N]
    i = i + 1

In [69]:
df = df.sort_values(by='rate_uniq', ascending=False)
pd.set_option('display.max_colwidth', 60)
pd.options.display.float_format = '{:,.4f}'.format

In [70]:
print(df)

                                                       term n_val  rate_val n_uniq  rate_uniq
44              `http://rs.tdwg.org/dwc/terms/occurrenceID`  2966    0.7688   2967     0.7691
7                                              `undefined0`  2966    0.7688   2967     0.7691
11            `http://rs.tdwg.org/dwc/terms/scientificName`  3858    1.0000    708     0.1835
20                    `http://purl.org/dc/terms/references`   892    0.2312    476     0.1234
3                    `http://rs.tdwg.org/dwc/terms/taxonID`   892    0.2312    476     0.1234
41      `http://plazi.org/terms/1.0/verbatimScientificName`   794    0.2058    455     0.1179
6             `http://gbif.org/dwc/terms/1.0/canonicalName`   794    0.2058    451     0.1169
33             `http://rs.tdwg.org/dwc/terms/catalogNumber`  1266    0.3281    442     0.1146
46                     `http://rs.tdwg.org/dwc/terms/genus`  3853    0.9987    322     0.0835
28  `http://rs.tdwg.org/dwc/terms/scientificNameAuthorship` 

In [71]:
dataset = dataset.fillna('')
dataset.select( "`http://rs.tdwg.org/dwc/terms/occurrenceID`" ).show(5)
# `http://rs.tdwg.org/dwc/terms/occurrenceID`
# An identifier for the Occurrence (as opposed to a particular digital record of the occurrence). In the absence 
# of a persistent global unique identifier, construct one from a combination of identifiers in the record that 
# will most closely make the occurrenceID globally unique.

+-----------------------------------------+
|http://rs.tdwg.org/dwc/terms/occurrenceID|
+-----------------------------------------+
|                                 33-37061|
|                           244-37465-1175|
|                                344-37463|
|                           545-37479-1187|
|                                742-37454|
+-----------------------------------------+
only showing top 5 rows



In [72]:
dataset.select( "`undefined0`" ).show(5)

+--------------+
|    undefined0|
+--------------+
|      33-37061|
|244-37465-1175|
|     344-37463|
|545-37479-1187|
|     742-37454|
+--------------+
only showing top 5 rows



In [73]:
dataset.select( "`http://rs.tdwg.org/dwc/terms/scientificName`" ).show(5, truncate = False)

+----------------------------------------------------+
|http://rs.tdwg.org/dwc/terms/scientificName         |
+----------------------------------------------------+
|Acestrorhynchus gr. lacustris (Lütken 1875)         |
|Charax sp. “Madeira”                                |
|Ageneiosus inermis (Linnaeus 1766)                  |
|Cyphocharax spiluropsis (Eigenmann & Eigenmann 1889)|
|Moenkhausia dichroura (Kner 1858)                   |
+----------------------------------------------------+
only showing top 5 rows



In [74]:
dataset.select( "`http://purl.org/dc/terms/references`" ).distinct().show(5, truncate = False)
# `http://purl.org/dc/terms/references`
# A related resource that is referenced, cited, or otherwise pointed to by the described resource.

+--------------------------------------------------------------+
|http://purl.org/dc/terms/references                           |
+--------------------------------------------------------------+
|http://treatment.plazi.org/id/F72087F4FFDDFFC43F94048A83903E5E|
|http://treatment.plazi.org/id/F72087F4FFD6FFCF3F94044B816A3FA0|
|http://treatment.plazi.org/id/03FD87FEFFFC353CFF655C111F8EF858|
|http://treatment.plazi.org/id/F72087F4FFC8FFD13F94065E80C53D53|
|http://treatment.plazi.org/id/F72087F4FFE1FFF83F94065E81EF3D52|
+--------------------------------------------------------------+
only showing top 5 rows



In [75]:
dataset.select( "`http://rs.tdwg.org/dwc/terms/taxonID`" ).distinct().show(5, truncate = False)
# `http://rs.tdwg.org/dwc/terms/taxonID`
# An identifier for the set of taxon information (data associated with the Taxon class). May be a global unique identifier or an identifier specific to the data set.

+--------------------------------------+
|http://rs.tdwg.org/dwc/terms/taxonID  |
+--------------------------------------+
|F72087F4FFE1FFF83F94030E80923883.taxon|
|F72087F4FFE4FFFC3F94053883D539FA.taxon|
|F72087F4FFCDFFD43F94047580BA3FB6.taxon|
|F72087F4FFCCFFD53F940126813F3AEB.taxon|
|092A87F7FFCCFE0851DEA2898991FF48.taxon|
+--------------------------------------+
only showing top 5 rows



In [ ]:
# http://plazi.org/terms/1.0/combinationAuthors  -> Page Not Found





In [99]:
from pyspark.sql.functions import concat_ws, col
dataset_id = dataset.select('`http://rs.tdwg.org/dwc/terms/scientificName`', 
                            '`http://rs.tdwg.org/dwc/terms/eventDate`', 
                            '`http://rs.tdwg.org/dwc/terms/locality`'
                           ).fillna('').toDF('sc_name','date', 'locality')
dataset_id.show(5, False)

+----------------------------------------------------+--------+---------------------------+
|sc_name                                             |date    |locality                   |
+----------------------------------------------------+--------+---------------------------+
|Acestrorhynchus gr. lacustris (Lütken 1875)         |19/06/01|Manuripi lake              |
|Charax sp. “Madeira”                                |28/07/02|Itenez-Blanco lake         |
|Ageneiosus inermis (Linnaeus 1766)                  |26/07/02|Itenez-Blanco river channel|
|Cyphocharax spiluropsis (Eigenmann & Eigenmann 1889)|11/08/02|Itenez lake                |
|Moenkhausia dichroura (Kner 1858)                   |17/07/02|Itenez-San Antonio lake    |
+----------------------------------------------------+--------+---------------------------+
only showing top 5 rows



In [103]:
dataset_id_concat = dataset_id.withColumn("concat", concat_ws("|", *dataset_id.columns) )
dataset_id_concat.select("concat").show(5, False)

+-------------------------------------------------------------------------+
|concat                                                                   |
+-------------------------------------------------------------------------+
|Acestrorhynchus gr. lacustris (Lütken 1875)|19/06/01|Manuripi lake       |
|Charax sp. “Madeira”|28/07/02|Itenez-Blanco lake                         |
|Ageneiosus inermis (Linnaeus 1766)|26/07/02|Itenez-Blanco river channel  |
|Cyphocharax spiluropsis (Eigenmann & Eigenmann 1889)|11/08/02|Itenez lake|
|Moenkhausia dichroura (Kner 1858)|17/07/02|Itenez-San Antonio lake       |
+-------------------------------------------------------------------------+
only showing top 5 rows



In [104]:
n_distinct_concat = dataset_id_concat.select( "concat" ).distinct().count()
print("Number of different values: " + str(n_distinct_concat) + "(from 3858)")

Number of different values: 3441(from 3858)


In [113]:
dataset_id_n = dataset_id.groupBy('sc_name','date', 'locality').count().select('sc_name','date', 'locality', col('count').alias('n'))
dataset_id_n.where(dataset_id_n.n > 1).orderBy(['sc_name','date']).show(10, False)

+-----------------------------------+----+--------+---+
|sc_name                            |date|locality|n  |
+-----------------------------------+----+--------+---+
|Abascantus grandis Becker 1977     |    |        |2  |
|Abascantus lobatus Stal 1864       |    |        |2  |
|Ablaptus amazonus Stal 1864        |    |        |2  |
|Ablaptus varicornis Walker 1867    |    |        |2  |
|Adoxoplatys willineri Kormilev 1949|    |        |2  |
|Agaclitus dromedarius Stal 1864    |    |        |2  |
|Agaclitus fallenii Stal 1864       |    |        |2  |
|Agrilus (Agrilus) caquetai         |    |        |2  |
|Agrilus (Agrilus) florae           |    |        |2  |
|Agroecus griseus Dallas 1851       |    |        |2  |
+-----------------------------------+----+--------+---+
only showing top 10 rows



In [118]:
dataset.filter(dataset["`http://rs.tdwg.org/dwc/terms/scientificName`"]=="Abascantus grandis Becker 1977").select(
    '`http://rs.tdwg.org/dwc/terms/scientificName`', 
    '`http://rs.tdwg.org/dwc/terms/eventDate`', 
    '`http://rs.tdwg.org/dwc/terms/locality`',
    '`http://rs.tdwg.org/dwc/terms/decimalLatitude`',
    '`http://rs.tdwg.org/dwc/terms/country`'
    ).fillna('').toDF('sc_name','date', 'locality', 'latitude', 'country').show(3, False)

+------------------------------+----+--------+--------+-------+
|sc_name                       |date|locality|latitude|country|
+------------------------------+----+--------+--------+-------+
|Abascantus grandis Becker 1977|    |        |        |       |
|Abascantus grandis Becker 1977|    |        |        |       |
+------------------------------+----+--------+--------+-------+



In [119]:
dataset.filter(dataset["`http://rs.tdwg.org/dwc/terms/scientificName`"]=="Abascantus grandis Becker 1977").show(3, False)

+--------------------------------------+--------------------------------------------+----------------------------------+--------------------------------------+------------------------------------+---------------------------------------+-------------------------------------------+----------+------------------------------------------+------------------------------------------+------------------------------------------+-------------------------------------------+--------------------------------------------+--------------------------------------+--------------------------------------+------------------------------------------------+---------------------------------------+--------------------------------------+--------------------------------------------+-----------------------------------+--------------------------------------------------------------+------------------------------------------------+--------------------------------------------+------------------------------------+----------